In [87]:
!pip install xgboost
!pip install --upgrade setuptools
!pip install pycld2-0.41-cp39-cp39-win_amd64.whl

Processing c:\users\alvin\desktop\ir_pro_jolene\pycld2-0.41-cp39-cp39-win_amd64.whl
pycld2 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [88]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
import time
#from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [89]:
!pip install nlpaug

In [90]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [91]:
!pip install textattack

In [92]:
from textattack.augmentation import EmbeddingAugmenter

In [93]:
# from google.colab import drive

# drive.mount('/content/drive')

In [94]:
df = pd.read_csv('label_dataset.csv', lineterminator='\n')

df.head()

Datetime  Quarter  Likes                    NFT  \
0  2022-06-27       22      2  Mutant Ape Yacht Club   
1  2023-02-11       31      2  Mutant Ape Yacht Club   
2  2022-12-30       24      0  Mutant Ape Yacht Club   
3  2022-12-29       24      0  Mutant Ape Yacht Club   
4  2022-12-29       24      1  Mutant Ape Yacht Club   

                                                Text  \
0  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...   
1  🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...   
2  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...   
3  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...   
4  Mutant Ape Yacht Club #27908 sold for 17 ETH (...   

                                          Clean Text vader textblob  \
0          mutant ape yacht club mayc nft sold eth k   neu      neu   
1  mutant ape yacht club bought floor h chg floor...   neu      neu   
2          mutant ape yacht club mayc nft sold eth k   neu      neu   
3          mutant ape yacht club mayc nft sold eth k   neu      neu   
4  mutant ape yacht club sold eth nft collection ...   neu      neu   

  Polarity (Reviewer 1) Polarity (Reviewer 2)\r  
0                   pos                   pos\r  
1                   pos                   pos\r  
2                   pos                   pos\r  
3                   pos                   pos\r  
4                   pos                   pos\r

In [95]:
df.rename(columns={'Polarity (Reviewer 1)':'label'}, inplace=True)

df.head()

Datetime  Quarter  Likes                    NFT  \
0  2022-06-27       22      2  Mutant Ape Yacht Club   
1  2023-02-11       31      2  Mutant Ape Yacht Club   
2  2022-12-30       24      0  Mutant Ape Yacht Club   
3  2022-12-29       24      0  Mutant Ape Yacht Club   
4  2022-12-29       24      1  Mutant Ape Yacht Club   

                                                Text  \
0  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...   
1  🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...   
2  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...   
3  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...   
4  Mutant Ape Yacht Club #27908 sold for 17 ETH (...   

                                          Clean Text vader textblob label  \
0          mutant ape yacht club mayc nft sold eth k   neu      neu   pos   
1  mutant ape yacht club bought floor h chg floor...   neu      neu   pos   
2          mutant ape yacht club mayc nft sold eth k   neu      neu   pos   
3          mutant ape yacht club mayc nft sold eth k   neu      neu   pos   
4  mutant ape yacht club sold eth nft collection ...   neu      neu   pos   

  Polarity (Reviewer 2)\r  
0                   pos\r  
1                   pos\r  
2                   pos\r  
3                   pos\r  
4                   pos\r

In [96]:
df['label'] = df['label'].replace('neg',0)

In [97]:
df['label'] = df['label'].replace('pos',1)

In [98]:
df['label'] = df['label'].replace('neu',2)

In [99]:
df.head()

Datetime  Quarter  Likes                    NFT  \
0  2022-06-27       22      2  Mutant Ape Yacht Club   
1  2023-02-11       31      2  Mutant Ape Yacht Club   
2  2022-12-30       24      0  Mutant Ape Yacht Club   
3  2022-12-29       24      0  Mutant Ape Yacht Club   
4  2022-12-29       24      1  Mutant Ape Yacht Club   

                                                Text  \
0  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...   
1  🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...   
2  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...   
3  🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...   
4  Mutant Ape Yacht Club #27908 sold for 17 ETH (...   

                                          Clean Text vader textblob  label  \
0          mutant ape yacht club mayc nft sold eth k   neu      neu    1.0   
1  mutant ape yacht club bought floor h chg floor...   neu      neu    1.0   
2          mutant ape yacht club mayc nft sold eth k   neu      neu    1.0   
3          mutant ape yacht club mayc nft sold eth k   neu      neu    1.0   
4  mutant ape yacht club sold eth nft collection ...   neu      neu    1.0   

  Polarity (Reviewer 2)\r  
0                   pos\r  
1                   pos\r  
2                   pos\r  
3                   pos\r  
4                   pos\r

In [100]:
df['label'].value_counts()

1.0    1053
2.0     453
0.0      95
Name: label, dtype: int64

In [101]:
df = df[['Clean Text','label']]

df.head()

Clean Text  label
0          mutant ape yacht club mayc nft sold eth k    1.0
1  mutant ape yacht club bought floor h chg floor...    1.0
2          mutant ape yacht club mayc nft sold eth k    1.0
3          mutant ape yacht club mayc nft sold eth k    1.0
4  mutant ape yacht club sold eth nft collection ...    1.0

In [102]:
df.isnull().values.any()

True

In [103]:
df = df.dropna()

In [104]:
df.isnull().values.any()

False

In [105]:
X_train, X_test, y_train, y_test = train_test_split(df['Clean Text'], df['label'], test_size=0.2,random_state=42)

In [106]:
y_train.value_counts()

1.0    848
2.0    353
0.0     79
Name: label, dtype: int64

In [107]:
vectorizer = TfidfVectorizer(min_df = 1,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [108]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [109]:
embed_aug = EmbeddingAugmenter()

In [110]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps3=embed_aug.augment(X_train[i])
    for sent in temps3:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [111]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1359,)
(1359,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_16800\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_16800\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [112]:
y_train.value_counts()

1.0    848
2.0    353
0.0    158
dtype: int64

In [113]:
aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)

In [114]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps1=aug.augment(X_train[i],n=3)
    for sent in temps1:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [115]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1833,)
(1833,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_16800\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_16800\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [116]:
y_train.value_counts()

1.0    848
0.0    632
2.0    353
dtype: int64

In [117]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [118]:
#XGB_classifier = GradientBoostingClassifier()
XGB_classifier = XGBClassifier()

XGB_classifier.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [119]:
t0 = time.time()
#XGB_classifier = GradientBoostingClassifier().fit(X_train_vectorized,y_train)
XGB_classifier = XGBClassifier(seed=0).fit(X_train_vectorized,y_train)
t1 = time.time()

time_linear_train = t1-t0

In [120]:
print("Training time: %fs" % (time_linear_train))

Training time: 28.083739s


In [121]:
y_pred=XGB_classifier.predict(X_test_vectorized)

In [122]:
print(accuracy_score(y_test, y_pred))

0.8286604361370716


In [123]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.62      0.65        16
         1.0       0.87      0.89      0.88       205
         2.0       0.76      0.73      0.74       100

    accuracy                           0.83       321
   macro avg       0.77      0.75      0.76       321
weighted avg       0.83      0.83      0.83       321



In [124]:
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

In [125]:
# Define the search space
param_grid = { 
    "learning_rate": [0.1, 0.01],
    "max_depth": [3, 5, 7, 9],
    "n_estimators": [50, 200, 300, 400, 500],
    "min_child_weight": [3, 5, 7, 9],
    "subsample": [0.3, 0.5, 0.8],
    "colsample_bytree": [0.3, 0.5 , 0.8]
    }
# Set up score
scoring = ['f1_macro']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [126]:
# Define grid search
grid_search = GridSearchCV(estimator=XGB_classifier, 
                           param_grid=param_grid, 
                           scoring='f1_macro', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(X_train_vectorized,y_train)
# Print grid search summary
grid_result
# Print the best score and the corresponding hyperparameters
print(f'The best score is {grid_result.best_score_:.4f}')
#print('The best score standard deviation is', round(grid_result.cv_results_['std_test_recall'][grid_result.best_index_], 4))
print(f'The best hyperparameters are {grid_result.best_params_}')

The best score is 0.8236
The best hyperparameters are {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 500, 'subsample': 0.8}


In [127]:
# Make prediction using the best model
grid_predict = grid_search.predict(X_test_vectorized)
# Get predicted probabilities
#grid_predict_prob = grid_search.predict_proba(X_test_vectorized)[:,1]

In [128]:
print(accuracy_score(y_test, y_pred))

0.8286604361370716


In [129]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.62      0.65        16
         1.0       0.87      0.89      0.88       205
         2.0       0.76      0.73      0.74       100

    accuracy                           0.83       321
   macro avg       0.77      0.75      0.76       321
weighted avg       0.83      0.83      0.83       321

